# Devoir de Calcul Différentiel : Courbes de niveau

## Préambule

### Dépendances logicielles

### Différentiation automatique

In [1]:
def grad(f):
    g = autograd.grad
    def grad_f(x, y):
        return np.array([g(f, 0)(x, y), g(f, 1)(x, y)])
    return grad_f

In [2]:
def J(f):
    j = autograd.jacobian
    def J_f(x, y):
        return np.array([j(f, 0)(x, y), j(f, 1)(x, y)]).T
    return J_f

## Ensemble de niveaux

### Analyse des ensembles de niveaux

#### Question 1

Soit $c \in \mathbb{R}$, l'ensemble $\{c\}$ est un fermé. La fonction $f$ étant continue, la courbe de niveau $c$, qui est en fait $f^-1(\{c\})$ est également un fermé.

#### Question 2

On remarque que le vecteur $T = \frac {1} {\|\nabla f(x_0)\|} \begin{pmatrix} \partial_2 f(x_0) \\ -\partial_1 f(x_0) \end{pmatrix}$ est le vecteur normalisé orthogonal au gradient donc tangent à la courbe au point $x_0$. Ainsi, $p(x_1,x_2)$ correspond au produit scalaire entre $T$ et le vecteur $MM_0 = \begin{pmatrix} x_1 - x_{10} \\ x_2 - x_{20} \end{pmatrix}$, ce qui géométriquement correspond à la projection orthogonale de $MM_0$ sur  $T$. 

#### Question 3